In [8]:
import cv2
import numpy as np
import os

def rotate_image(image, angle):
    # Get the image dimensions (height, width)
    height, width = image.shape[:2]
    
    # Calculate the center of the image
    center = (width // 2, height // 2)
    
    # Get the rotation matrix using cv2.getRotationMatrix2D
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)  # angle, scale
    
    # Perform the rotation
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    
    return rotated_image

# Example usage
image_path = 'test_v2/test_0.jpeg'  # Input image path
output_path = 'test_v2_rotated/test_0.png'  # Output image path

# Check if the image is loaded successfully
image = cv2.imread(image_path)
if image is None:
    print(f"Error: Unable to load image at {image_path}")
else:
    # Rotate the image by 10 degrees
    rotated_image = rotate_image(image, 3)
    
    # Ensure the output folder exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # Save the rotated image
    if cv2.imwrite(output_path, rotated_image):
        print(f"Image saved successfully to {output_path}")
    else:
        print("Error: Unable to save the rotated image.")

Image saved successfully to test_v2_rotated/test_0.png


In [11]:
import cv2
import numpy as np
import glob
import os

def align_image(training_image, reference_image):
    # Convert images to grayscale
    gray_train = cv2.cvtColor(training_image, cv2.COLOR_BGR2GRAY)
    gray_test = cv2.cvtColor(reference_image, cv2.COLOR_BGR2GRAY)

    # ORB Detector
    orb = cv2.ORB_create()
    keypoints1, descriptors1 = orb.detectAndCompute(gray_train, None)
    keypoints2, descriptors2 = orb.detectAndCompute(gray_test, None)

    # Match features using Brute Force Matcher
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(descriptors1, descriptors2)

    # Sort matches by distance (best matches first)
    matches = sorted(matches, key=lambda x: x.distance)

    # Extract matched keypoints
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Compute the affine transformation
    M, _ = cv2.estimateAffinePartial2D(src_pts, dst_pts)

    # Apply the transformation to align the image
    aligned_image = cv2.warpAffine(training_image, M, (reference_image.shape[1], reference_image.shape[0]))
    return aligned_image

# Load a reference test image
test_img = cv2.imread("test_v2_rotated/test_0.png")

# Define the output folder
output_folder = "train_v2_rotated/"

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process all training images
training_images_paths = glob.glob("train_v2/*.jpeg")
for img_path in training_images_paths:
    train_img = cv2.imread(img_path)
    aligned_img = align_image(train_img, test_img)
    
    # Get the filename from the image path, replace extension with .png, and save the aligned image
    output_path = os.path.join(output_folder, os.path.splitext(os.path.basename(img_path))[0] + ".png")
    cv2.imwrite(output_path, aligned_img)

    print(f"Saved aligned image to {output_path}")

Saved aligned image to train_v2_rotated/train_0.png
Saved aligned image to train_v2_rotated/train_1.png
Saved aligned image to train_v2_rotated/train_2.png
Saved aligned image to train_v2_rotated/train_3.png
Saved aligned image to train_v2_rotated/train_4.png
Saved aligned image to train_v2_rotated/train_5.png


In [16]:
import cv2
import os

def crop_image(image, crop_top, crop_bottom, crop_left, crop_right):
    # Get image dimensions
    img_height, img_width = image.shape[:2]
    
    # Calculate the crop box using the specified crop amounts
    x1 = crop_left
    y1 = crop_top
    x2 = img_width - crop_right
    y2 = img_height - crop_bottom
    
    # Ensure the crop box is within the image bounds
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(img_width, x2), min(img_height, y2)
    
    # Crop and return the image
    cropped_image = image[y1:y2, x1:x2]
    return cropped_image

def batch_process_images(input_folder, output_folder, crop_top, crop_bottom, crop_left, crop_right):
    # Get list of all files in the directory
    files = os.listdir(input_folder)
    
    # Process each image file
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check if it's an image
            image_path = os.path.join(input_folder, file)
            image = cv2.imread(image_path)  # Read the image
            
            # Crop the image
            cropped_image = crop_image(image, crop_top, crop_bottom, crop_left, crop_right)
            
            # Save the cropped image
            output_path = os.path.join(output_folder, file)
            cv2.imwrite(output_path, cropped_image)

# Example usage
input_folder = 'test_v2_rotated'  # Input folder path containing images
output_folder = 'test_v2_cropped'  # Output folder to save cropped images
crop_top, crop_bottom, crop_left, crop_right = 250, 250, 400, 550  # Cropping values for each side

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

batch_process_images(input_folder, output_folder, crop_top, crop_bottom, crop_left, crop_right)

In [17]:
import cv2
import os

def crop_image(image, crop_top, crop_bottom, crop_left, crop_right):
    # Get image dimensions
    img_height, img_width = image.shape[:2]
    
    # Calculate the crop box using the specified crop amounts
    x1 = crop_left
    y1 = crop_top
    x2 = img_width - crop_right
    y2 = img_height - crop_bottom
    
    # Ensure the crop box is within the image bounds
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(img_width, x2), min(img_height, y2)
    
    # Crop and return the image
    cropped_image = image[y1:y2, x1:x2]
    return cropped_image

def batch_process_images(input_folder, output_folder, crop_top, crop_bottom, crop_left, crop_right):
    # Get list of all files in the directory
    files = os.listdir(input_folder)
    
    # Process each image file
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # Check if it's an image
            image_path = os.path.join(input_folder, file)
            image = cv2.imread(image_path)  # Read the image
            
            # Crop the image
            cropped_image = crop_image(image, crop_top, crop_bottom, crop_left, crop_right)
            
            # Save the cropped image
            output_path = os.path.join(output_folder, file)
            cv2.imwrite(output_path, cropped_image)

# Example usage
input_folder = 'train_v2_rotated'  # Input folder path containing images
output_folder = 'train_v2_cropped'  # Output folder to save cropped images
crop_top, crop_bottom, crop_left, crop_right = 250, 250, 400, 550  # Cropping values for each side

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

batch_process_images(input_folder, output_folder, crop_top, crop_bottom, crop_left, crop_right)